## Imports

In [1]:
from nltk.corpus import stopwords
from wordcloud import WordCloud
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

2025-05-26 19:47:46.931294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748288867.151500      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748288867.215614      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Data loading

In [2]:
df = pd.read_csv("/kaggle/input/dsaa-6100-movie-review-sentiment-classification/movie_reviews/movie_reviews.csv")
df.head()

,text,label
0,If you havent seen this movie than you need to...,1
1,but Cinderella gets my vote not only for the w...,0
2,This movie is pretty cheesy but I do give it c...,1
3,I have not seen a Van Damme flick for a while ...,1
4,This is a sleeper It defines Nicholas Cage The...,1


In [3]:
dataset = Dataset.from_pandas(df)

## Pretrained  distilbert/distilbert-base-uncased-finetuned-sst-2-english

In [22]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis", truncation=True)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [23]:
reviews = ["Best film", "worst film"]
pipe(reviews)

[{'label': 'POSITIVE', 'score': 0.9997945427894592},
 {'label': 'NEGATIVE', 'score': 0.9997544884681702}]

## Submit

In [16]:
test = pd.read_csv("/kaggle/input/dsaa-6100-movie-review-sentiment-classification/test_data.csv")
test.head()

,Id,text
0,0,What can possibly said about this movie other ...
1,1,I dont care how many bad reviews purple rain g...
2,2,Ken Russell directed this weird Not very ero...
3,3,This is a great movie from the lost age of rea...
4,4,I have a problem with the movie snobs who cons...


In [17]:
test.shape

(10000, 2)

In [18]:
test.isna().sum()

Id      0
text    0
dtype: int64

In [24]:
results = pipe(test["text"].tolist())

In [25]:
results[:5]

[{'label': 'NEGATIVE', 'score': 0.9994348883628845},
 {'label': 'POSITIVE', 'score': 0.9991669654846191},
 {'label': 'NEGATIVE', 'score': 0.9982131719589233},
 {'label': 'POSITIVE', 'score': 0.9988514184951782},
 {'label': 'POSITIVE', 'score': 0.9854157567024231}]

In [27]:
maps = {
    "NEGATIVE": 0,
    "POSITIVE": 1,
}

pd.DataFrame({
    "Id": list(range(len(results))),
    "Category": [maps[x["label"]] for x in results]
}).to_csv("distilbert_pretrained_sst.csv", index=False)